## The Trainer API
<br>

- [The Trainer API](https://www.youtube.com/watch?v=nvBXf7s7vTI&list=PLo2EIpI_JMQvWfQndUesu0nPBAtZ9gP1o&index=25)


In [ ]:
%%capture
!pip install datasets

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")

In [ ]:
%%capture
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenizer_function(exemples):
  return tokenizer(exemples["sentence1"], exemples["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenizer_function, batched=True)
datacollator = DataCollatorWithPadding(tokenizer)

In [ ]:
%%capture
from transformers import AutoModelForSequenceClassification, TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/accelerate
training_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01

    )

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=datacollator,
    tokenizer=tokenizer
)

# trainer.train()

In [ ]:
# predictions = trainer.predict(tokenized_datasets["validation"])
# print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np
from datasets import load_metric

# metric = load_metric("glue", "mrpc")
# preds = np.argmax(predictions.predictions, axis=-1)
# metric.compute(predictions=preds, references=predictions.label_ids)